# Advent of Code 2021 - Day 20

In [280]:
enum class Pixel(val id: Char, val value: Int) {
    Dark('.', 0),
    Light('#', 1),
    ;
    companion object {
        private val idMap: Map<Char, Pixel> = Pixel.values().associateBy(Pixel::id)
        fun fromId(id: Char): Pixel = idMap[id]!!
    }
}

@JvmInline value class Algorithm(val pixels: List<Pixel>) {
    @JvmInline value class Index(val at: List<Pixel>)
}

typealias Image = List<List<Pixel>>

data class Index(val row: Int, val col: Int)

In [281]:
import java.io.File

val (algorithm: Algorithm, image: Image) = File("Day20.input.txt")
    .bufferedReader()
    .readLines()
    .let {
        Algorithm(it.first().map { Pixel.fromId(it) }) to
            it.drop(2).map { it.map { Pixel.fromId(it) } }
    }

## Part 1

Given an `image`, enhance it by finding each `Pixel`'s new `Pixel` within the given `algorithm`. A `Pixel`'s enhanced pixel can be found by taking all surrounding `Pixel`s from top-down to left-right and interpreting their joined `Pixel::value` as a binary integer. This binary integer can then be used as the index of the enhanced pixel within an `Algorithm`. All `Pixel`s within an `Image` must be enhanced simultaneously. How many `Pixel.Light`s are there after two enhancements?

In [282]:
fun Algorithm.Index.toInt(): Int = at.joinToString("") { it.value.toString() }.toInt(2)

fun Algorithm.pixelAt(index: Algorithm.Index): Pixel = pixels[index.toInt()]

In [283]:
class LightPixels(indices: Set<Index>) : Set<Index> by indices

fun Image.lightPixels() = LightPixels(
    image.foldIndexed(mutableSetOf()) { r, outer, row ->
        row.foldIndexed(outer) { c, inner, it -> 
            inner.apply { if (it == Pixel.Light) inner.add(Index(r,c)) }
        }
    }
)

fun LightPixels.affects(): Set<Index> {
    return fold(mutableSetOf<Index>()) { acc, index ->
        (-2..2).forEach { r -> (-2..2).forEach { c->
            acc.add(Index(index.row + r, index.col + c))
        } }
        acc
    }
}

fun LightPixels.enhanceBy(algorithm: Algorithm): LightPixels {
    fun Index.toAlgorithmIndex() = Algorithm.Index(
        (-1..1).flatMap { r -> (-1..1).map { c -> 
            if (Index(this.row + r, this.col + c) in this@enhanceBy) Pixel.Light
            else Pixel.Dark 
        } }
    )

    return LightPixels(
        affects().fold(mutableSetOf<Index>()) { acc, it ->
            if (algorithm.pixelAt(it.toAlgorithmIndex()) == Pixel.Light) acc.add(it)
            acc
        }
    )
}

fun LightPixels.enhanceBy(algorithm: Algorithm, times: Int): LightPixels {
    return (1..times).fold(this) { acc, _ ->
        acc.enhanceBy(algorithm)
    }
}

fun LightPixels.print(): LightPixels {
    val minRow = minOf { it.row }; val maxRow = maxOf { it.row }
    val minCol = minOf { it.col }; val maxCol = maxOf { it.col }
    
    (minRow..maxRow).forEach { r ->
        (minCol..maxCol).forEach { c ->
            if (Index(r, c) in this) print(Pixel.Light.id)
            else print(Pixel.Dark.id)
        }
        println()
    }
    println()
    return this
}

### Notes

So I originally was thinking that the "infinite" nature of this problem meant that the image would be large and sparely "lit". This meant that I should only keep track of only `Pixel.Light`s since if it's not lit, then it was a `Pixel.Dark`. I could then figure out which pixels to process by taking a 5x5 box around the pixel. However this made some assumptions that did not hold true in the actual input. The actual input's zeroth index was lit and the last index was dark. This meant that "infinite" part of the problem would actually flip back and forth rather than staying dark until it was "affected" by a lit pixel. This took me a long time to figure out.

## Part 1 - Round 2

In [284]:
fun Image.print(): Image = also {
    forEach { it.map(Pixel::id).forEach(::print).also { println() } }
}

operator fun Image.contains(index: Index): Boolean = index.row in indices && index.col in this[index.row].indices

fun Image.getOrDefault(index: Index, default: Pixel) =
    if (index in this) this[index.row][index.col] else default

fun Image.enhanceBy(algorithm: Algorithm, default: Pixel): Image {
    fun Index.toAlgorithmIndex() = Algorithm.Index(
        (-1..1).flatMap { r -> (-1..1).map { c ->  getOrDefault(Index(row + r, col + c), default) } }
    )
    
    return (-1..size).map { row ->
        (-1..this[0].size).map { col ->
            algorithm.pixelAt(Index(row, col).toAlgorithmIndex())
        }
    }
}

fun Image.enhanceBy(algorithm: Algorithm, times: Int): Image =
    (1..times).fold(this) { acc, it ->
        acc.enhanceBy(
            algorithm,
            if (it % 2 == 1) algorithm.pixels.last() 
            else algorithm.pixels.first()
        )
    }

fun Image.count(pixel: Pixel) = sumOf { row -> row.count { it == pixel } }

image.enhanceBy(algorithm, 2).count(Pixel.Light)

5597

### Notes

This approach was a lot more straight forward. We can "expand" the area around our image by mapping an extra row and extra column on each side of the image. To deal with the "infinite" nature of the `Image` we can pass in a default value to `Image::getOrDefault` to allow us to threat how "unmapped" values should be handled. The flipping is resolved by using the last algorithm pixel on every odd enhancement and the first algorithm pixel on every even enhancement.

## Part 2

How many `Pixel.Light`s in an `image` enhanced 50 times?

In [285]:
image.enhanceBy(algorithm, 50).count(Pixel.Light)

18723

### Notes

Same algorithm, just 50 times.